<center><h1 style="font-size:40px;">Project 2<br> GANs
</h1></center>

In [1]:
%load_ext autoreload
%autoreload 2
# Hacky solution to acces the global utils package
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('../..')))

1) We first start with importing all required modules

In [60]:
import pytorch_lightning as pl
import torch.nn as nn
from config import LabConfig

cfg = LabConfig()

import numpy as np

import torchvision
import matplotlib.pyplot as plt
import torch

In [45]:
# Augmentation

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH),interpolation=torchvision.transforms.InterpolationMode.NEAREST), 
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH),interpolation=torchvision.transforms.InterpolationMode.NEAREST), #, 
])


2) Here, we have the network model class definition. In this class, the most important functions are **build_generator()** and **build_discriminator()**. As defined in the exercises section, your task is to update the both network architectures defined in these functions.

In [11]:
# Generator Model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(100, 128)
        self.fc2 = nn.Linear(128, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, 28*28)
        self.bn1 = nn.BatchNorm1d(128)
        self.bn2 = nn.BatchNorm1d(512)
        self.bn3 = nn.BatchNorm1d(1024)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.leaky_relu(x, 0.2)
        # mixing above 3 steps in single line
        x = F.leaky_relu(self.bn2(self.fc2(x)), 0.2)
        x = F.leaky_relu(self.bn3(self.fc3(x)), 0.2)
        x = torch.tanh(self.fc4(x))
        return x.view(x.shape[0], *img_shape)


# Discriminator Model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 1)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = torch.sigmoid(self.fc4(x))
        return x

In [12]:
class GAN(pl.LightningModule):
    
    def __init__(self, **hparams):
        super(GAN, self).__init__()

        self.hparams.update(hparams)
        self.generator = Generator()
        self.discriminator = Discriminator()
    
    # Forward Pass of Model
    def forward(self, x):
        return self.discriminator(x)
    
    # Loss Function
    def loss_function(self, y_hat, y):
        return nn.BCELoss()(y_hat, y)
    
    # Optimizers
    def configure_optimizers(self):
        optimizer_G = torch.optim.Adam(self.generator.parameters(), lr=self.hparams.lr, betas=(0.4, 0.999))
        optimizer_D = torch.optim.Adam(self.discriminator.parameters(), lr=self.hparams.lr, betas=(0.4, 0.999))
        
        # return the list of optimizers and second empty list is for schedulers (if any)
        return [optimizer_G, optimizer_D], []
    
    # Training Loop
    def training_step(self, batch, batch_idx, optimizer_idx):
        # batch returns x and y tensors
        real_images, _ = batch
        
        # ground truth (tensors of ones and zeros) same shape as images
        valid = torch.ones(real_images.size(0), 1)
        fake = torch.zeros(real_images.size(0), 1)
        
        # svaing loss_function as local variable
        criterion = self.loss_function
        
        # As there are 2 optimizers we have to train for both using 'optimizer_idx'
        ## Generator
        if optimizer_idx == 0:
            # Generating Noise (input for the generator)
            gen_input = torch.randn(real_images.shape[0], 100)
            
            # Converting noise to images
            self.gen_images = self.generator(gen_input)
            
            # Calculating generator loss
            # How well the generator can create real images
            g_loss = criterion(self(self.gen_images), valid)
            
            # for output and logging purposes (return as dictionaries)
            
            self.log('g_loss_train', g_loss)
            return g_loss

        ## Discriminator
        if optimizer_idx == 1:
            # Calculating disciminator loss
            # How well discriminator identifies the real and fake images
            real_loss = criterion(self(real_images), valid)
            fake_loss = criterion(self(self.gen_images.detach()), fake)
            d_loss = (real_loss + fake_loss)/2.0
            
            # for output and logging purposes (return as dictionaries)
            self.log('d_loss_train',d_loss)
            return d_loss
    
    # calls after every epoch ends
    def on_epoch_end(self):
        # Saving 5x5 grid
        utils.save_image(self.gen_images.data[:25], path+'/%d.png' % self.current_epoch, nrow=5, padding=0, normalize=True)

In [14]:
from DL_labs.utils.utils import dict_to_args
hparams = dict_to_args({'batch_size': 32,
                        'lr': 2e-4,
                        'epochs': 20,
                        'optimizer':'Adam'
                      })

gan = GAN(hparams=hparams)

# PyTorch Lightning Trainer (where loss backward, optimizer grading, gpu/tpu code automates)
trainer = pl.Trainer(max_epochs=hparams.epochs, fast_dev_run=False)

# Fitting the model to trainer
trainer.fit(gan, datamodule=dataloader

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  warnings.warn(*args, **kwargs)


MisconfigurationException: No `train_dataloader()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.

## EXERCISES

#### Exercise 1)
Update the network architecture given in  **build_generator**  and  **build_discriminator**  of the class GANModel. Please note that the current image resolution is set to 32x32 (i.e. IMAGE_WIDTH and IMAGE_HEIGHT values) in the file configGAN.py. 
This way initial experiements can run faster. Once you implement the inital version of the network, please set the resolution values back to 128x128. Experimental results should be provided for this high resolution images.  

**Hint:** As a generator model, you can use the segmentation model implemented in lab03. Do not forget to adapt the input and output shapes of the generator model in this case.

#### Exercise 2) 
Use different **optimization** (e.g. ADAM, SGD, etc) and **regularization** (e.g. data augmentation, dropout) methods to increase the network accuracy. 